In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("D:/regression_project/Cab-Weather Data/cab_rides.csv")
data1 = pd.read_csv("D:/regression_project/Cab-Weather Data/weather.csv",sep=',')
cb = data.copy()
we = data1.copy()

In [2]:
cb.head(1)

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared


In [3]:
# Checking it is second or mili-second:
len(str(cb.loc[1,'time_stamp'])) # it is mili-second

13

In [4]:
we.head(1)

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.0,1012.14,0.1228,1545003901,0.77,11.25


In [5]:
#checking it is second or mili-second:
len(str(we.loc[1, 'time_stamp'])) # it is seconds


10

In [6]:
we['time_stamp'].dtype

dtype('int64')

In [7]:
#Changing time_stamp to date and time:
cb['date_time'] = pd.to_datetime(cb['time_stamp']/1000,unit='s')
we['date_time'] = pd.to_datetime(we['time_stamp'],unit='s')

In [8]:
cb['date_time'].unique()
we['date_time'].unique()

<DatetimeArray>
['2018-12-16 23:45:01', '2018-11-27 19:45:20', '2018-11-27 00:17:13',
 '2018-11-29 00:02:07', '2018-12-14 11:45:01', '2018-11-26 17:35:10',
 '2018-12-03 09:52:59', '2018-12-15 10:45:01', '2018-12-14 18:45:01',
 '2018-12-17 17:45:01',
 ...
 '2018-11-28 21:02:09', '2018-12-02 22:52:54', '2018-11-26 09:05:11',
 '2018-11-26 06:40:30', '2018-11-30 11:52:54', '2018-12-01 09:52:54',
 '2018-12-17 13:45:02', '2018-12-02 11:52:54', '2018-12-17 22:45:06',
 '2018-12-03 06:52:54']
Length: 840, dtype: datetime64[ns]

In [9]:
cb['source'].count()
we['location'].count()

np.int64(6276)

In [10]:
cb['source'].unique()
we['location'].unique()

array(['Back Bay', 'Beacon Hill', 'Boston University', 'Fenway',
       'Financial District', 'Haymarket Square', 'North End',
       'North Station', 'Northeastern University', 'South Station',
       'Theatre District', 'West End'], dtype=object)

In [11]:
# Creating common column for merging:
cb['m_date'] = cb['source'].astype(str) + "-" +cb['date_time'].dt.date.astype(str) + "-" +cb['date_time'].dt.hour.astype(str)
we['m_date'] = we['location'].astype(str) + "-" +we['date_time'].dt.date.astype(str) + "-" +we['date_time'].dt.hour.astype(str)

In [12]:
# Remove duplicates:
we = we.drop_duplicates(subset='m_date')

In [13]:
# Merging the datasets:
cab = cb.merge(we,on='m_date',how='left')

In [14]:
cab.count()

distance            693071
cab_type            693071
time_stamp_x        693071
destination         693071
source              693071
price               637976
surge_multiplier    693071
id                  693071
product_id          693071
name                693071
date_time_x         693071
m_date              693071
temp                690107
location            690107
clouds              690107
pressure            690107
rain                 98756
time_stamp_y        690107
humidity            690107
wind                690107
date_time_y         690107
dtype: int64

In [15]:
cab.isnull().sum()

distance                 0
cab_type                 0
time_stamp_x             0
destination              0
source                   0
price                55095
surge_multiplier         0
id                       0
product_id               0
name                     0
date_time_x              0
m_date                   0
temp                  2964
location              2964
clouds                2964
pressure              2964
rain                594315
time_stamp_y          2964
humidity              2964
wind                  2964
date_time_y           2964
dtype: int64

In [16]:
# Remove null value:
cab = cab.dropna(subset='price')

# Replace the null values in balance columns:
weather_cols = ['temp', 'clouds', 'humidity', 'wind','pressure']
cab[weather_cols] = cab[weather_cols].fillna(method='ffill').fillna(method='bfill')

cab['rain'] = cab['rain'].fillna(0)

cab['location'] = cab['location'].fillna(cab['source'])


C:\Users\ahamm\AppData\Local\Temp\ipykernel_23372\1029848516.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cab[weather_cols] = cab[weather_cols].fillna(method='ffill').fillna(method='bfill')


In [17]:
cab.columns

Index(['distance', 'cab_type', 'time_stamp_x', 'destination', 'source',
       'price', 'surge_multiplier', 'id', 'product_id', 'name', 'date_time_x',
       'm_date', 'temp', 'location', 'clouds', 'pressure', 'rain',
       'time_stamp_y', 'humidity', 'wind', 'date_time_y'],
      dtype='object')

In [18]:
# checking both columns information are same :
(cab['date_time_x'].dt.hour == cab['date_time_y'].dt.hour).mean()

np.float64(0.9957145723350095)

In [19]:
# Removing unwanted columns:
cab = cab.drop(columns=['time_stamp_x','time_stamp_y','date_time_y','m_date'],errors='ignore')
cab.columns

Index(['distance', 'cab_type', 'destination', 'source', 'price',
       'surge_multiplier', 'id', 'product_id', 'name', 'date_time_x', 'temp',
       'location', 'clouds', 'pressure', 'rain', 'humidity', 'wind'],
      dtype='object')

In [20]:
# Creating new features based on the date_time:
cab['hour'] = cab['date_time_x'].dt.hour
cab['day'] = cab['date_time_x'].dt.day
cab['month'] = cab['date_time_x'].dt.month
cab['weekday'] = cab['date_time_x'].dt.weekday
cab['weekend'] = cab['date_time_x'].dt.weekday.isin([5,6]).astype(int)

# Removing date_time after extraction:
cab = cab.drop(columns=['date_time_x'])

In [21]:
cab['id'].unique()
cab['product_id'].unique() 


array(['lyft_line', 'lyft_premier', 'lyft', 'lyft_luxsuv', 'lyft_plus',
       'lyft_lux', '6f72dfc5-27f1-42e8-84db-ccc7a75f6969',
       '6c84fd89-3f11-4782-9b50-97c468b19529',
       '55c66225-fbe7-4fd5-9072-eab1ece5e23e',
       '9a0e7b09-b92b-4c41-9779-2ad22b4d779d',
       '6d318bcc-22a3-4af6-bddd-b409bfce1546',
       '997acbb5-e102-41e1-b155-9df7de0a73f2'], dtype=object)

In [22]:
(cab['source']==cab['location']).all() # both are same or not 
cab = cab.drop(columns=['location','id','product_id'])

In [23]:
# Reseting the index:
cab.reset_index(inplace=True)
cab.index

RangeIndex(start=0, stop=637976, step=1)

In [24]:
# Renaming the column to better understading:
cab = cab.rename(columns={
    'distance': 'distance_km',
    'cab_type': 'company',
    'destination': 'drop_location',
    'source': 'pickup_location',
    'surge_multiplier': 'surge',
    'name': 'ride_type',
    'temp': 'temperature',
    'clouds': 'cloud_coverage',
    'rain': 'rainfall',
    'wind': 'wind_speed'
})


In [25]:
cab = cab.drop(columns=['index'])
cab.columns

Index(['distance_km', 'company', 'drop_location', 'pickup_location', 'price',
       'surge', 'ride_type', 'temperature', 'cloud_coverage', 'pressure',
       'rainfall', 'humidity', 'wind_speed', 'hour', 'day', 'month', 'weekday',
       'weekend'],
      dtype='object')

In [26]:
cab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637976 entries, 0 to 637975
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   distance_km      637976 non-null  float64
 1   company          637976 non-null  object 
 2   drop_location    637976 non-null  object 
 3   pickup_location  637976 non-null  object 
 4   price            637976 non-null  float64
 5   surge            637976 non-null  float64
 6   ride_type        637976 non-null  object 
 7   temperature      637976 non-null  float64
 8   cloud_coverage   637976 non-null  float64
 9   pressure         637976 non-null  float64
 10  rainfall         637976 non-null  float64
 11  humidity         637976 non-null  float64
 12  wind_speed       637976 non-null  float64
 13  hour             637976 non-null  int32  
 14  day              637976 non-null  int32  
 15  month            637976 non-null  int32  
 16  weekday          637976 non-null  int3

In [27]:
#Saving File to CSV:
cab.to_csv("Cab_price_cleaned",index=False)